

https://www.kaggle.com/christofhenkel/keras-baseline

In [ ]:
# # Install the plaidml backend
# ## DO THIS BEFORE IMPORTING KERAS OR TENSOR TO USE PLAIDML
# import plaidml.keras
# plaidml.keras.install_backend()

# # Help MacOS be able to use Keras
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [ ]:
# # Gets rid of the processor warning.
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# ## extra imports to set GPU options
# import tensorflow as tf
# from keras import backend as K
 
# ###################################
# # TensorFlow wizardry
# config = tf.ConfigProto()
 
# # Don't pre-allocate memory; allocate as-needed
# config.gpu_options.allow_growth = True
 
# # Only allow a total of half the GPU memory to be allocated
# # config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# # Create a session with the above options specified.
# K.tensorflow_backend.set_session(tf.Session(config=config))
# ###################################

In [ ]:
import tensorflow as tf
import keras
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.tensorflow_backend.set_session(sess)

In [ ]:
## Use this as callback on .fit (callbacks=[cb])
## NOT necessary

# better progress bars for jupyter and won't kill model fit
# Command line the following line to get tqdm installed properly
# python -m pip install tqdm
from keras_tqdm import TQDMNotebookCallback
# keras, model definition...
cb = TQDMNotebookCallback()
setattr(cb,'on_train_batch_begin',lambda x,y:None)
setattr(cb,'on_train_batch_end',lambda x,y:None)

In [ ]:
from __future__ import print_function
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Activation, Input, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Conv2DTranspose, UpSampling2D
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
import pandas as pd
import time
import glob
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

### Constants

In [ ]:
TRAIN_IMAGE_DIR = '../images/network_training/build/0/'
TRAIN_MASK_DIR = '../images/network_training/mask/0/'
TEST_IMAGE_DIR = '../images/network_training/test/0/'

train_names = glob.glob(TRAIN_IMAGE_DIR + '*')
train_fns = [os.path.basename(x) for x in train_names[0:50]]

In [ ]:
train_fns[:5]

### Image and Mask Preprocessing

In [ ]:
split_rows = 5
split_cols = 5
img_width = 4800
img_height = 4800


In [ ]:
def image_splitter(image, split_cols, split_rows):

    image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_AREA)
    width = image.shape[0] 
    height = image.shape[1]
    imglist = []

    for startpoint in np.linspace(0,width,split_cols, endpoint=False):
        endpoint=startpoint + (width / split_cols)

        for startp2 in np.linspace(0,height,split_rows, endpoint=False):
            endp2=startp2 + (width / split_rows)

            imglist.append(image[int(startp2):int(endp2), int(startpoint):int(endpoint)]
                        )
    return np.array(imglist)

In [ ]:
X = [np.array(image_splitter(cv2.imread(TRAIN_IMAGE_DIR + p).astype(np.uint8), split_cols, split_rows)) for p in tqdm(train_fns)]

In [ ]:
X = np.vstack(X)/255

In [ ]:
X.shape

In [ ]:
y = [np.array(image_splitter(cv2.imread(TRAIN_MASK_DIR + p).astype(np.uint8), split_cols, split_rows)) for p in tqdm(train_fns)]

In [ ]:
y = np.vstack(y)/255

In [ ]:
y.shape

### Train and Val Split and Modeling

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=23, test_size = 0.2)

In [ ]:
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_train: {X_val.shape}')
print(f'X_train: {y_val.shape}')

## U-Net Architecture

In [ ]:


inputs = Input(shape=(X_train.shape[1], X_train.shape[2], 3))

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(3, 1, activation = 'softmax')(conv9)

model = Model(input = inputs, output = conv10)

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()



In [ ]:
# from keras.models import load_model
 
# # load model
# model = load_model('keras_masks_boxes.h5')

In [ ]:
batch_size = None
epochs = 1
steps_per_epoch=X_train.shape[0] // 4

In [ ]:
early_stop = EarlyStopping(patience=5)
check_point = ModelCheckpoint('keras_masks_boxes.hdf5',save_best_only=True)

history = model.fit(X_train, 
                    y_train, 
                    epochs=epochs, 
                    validation_data=(X_val, y_val),
                    verbose=1, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=steps_per_epoch // 5,
                    callbacks=[early_stop, check_point, cb],
                    batch_size=batch_size)

In [ ]:
# save model and architecture to single file
model.save("keras_masks_boxes.h5")

## Testing

### Test Images Preprocessing

In [ ]:
test_names = glob.glob(TEST_IMAGE_DIR + '*')
test_fns = [os.path.basename(x) for x in test_names[0:1]]

In [ ]:
# X_test = [np.array(
#         cv2.imread(
#             TEST_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE)
#             [0:crop_height, 0:crop_width], 
#             dtype=np.uint8) for p in tqdm(test_fns)]

# X_test = np.array(X_test)/255
# X_test = np.expand_dims(X_test,axis=3)

In [ ]:
X_test = [np.array(
        cv2.imread(
            TEST_IMAGE_DIR + p)
            [0:crop_height, 0:crop_width], 
            dtype=np.uint8) for p in tqdm(test_fns)]

In [ ]:
X_test = np.array(X_test)/255

In [ ]:
# # Not necessary for color images
# X_test = np.expand_dims(X_test,axis=3)

In [ ]:
X_test.shape

### Predictions

In [ ]:
pred = model.predict(X_test, verbose = True)

In [ ]:
pred.shape

In [ ]:
pred.mean()

In [ ]:
pred

In [ ]:
model.evaluate(x=X_val, y=y_val, batch_size=16)

In [ ]:
model.evaluate(x=X_test, y=pred)

In [ ]:
pixel_image = pred[0].reshape(-1,1)

In [ ]:
pixel_image

In [ ]:
yes_pixels = []
for pixel in pixel_image:
    if pixel > .13:
        yes_pixels.append(1)
    else:
        yes_pixels.append(0)

In [ ]:
new_mask = np.array(yes_pixels, dtype=np.uint8).reshape(crop_height, crop_width)

In [ ]:
new_mask[0:1]

In [ ]:
plt.imshow(new_mask)

In [ ]:
test_image = cv2.imread('../images/network_training/test/0/'+test_fns[0])[0:crop_width, 0:crop_height]

In [ ]:
test_image.shape

In [ ]:
# test_image = np.array(X_test[0]).reshape(256,256)

In [ ]:
plt.imshow(test_image)

In [ ]:
print(new_mask.shape)
print(test_image.shape)
print(pixel_image.shape)

In [ ]:
print(new_mask.dtype)
print(test_image.dtype)
print(pixel_image.dtype)

In [ ]:
result = cv2.bitwise_and(test_image, test_image, mask=new_mask)

In [ ]:
result

In [ ]:
result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12,12))

plt.imshow(result)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(test_image)